All imports

In [14]:
import mediapipe as mp
import cv2
import numpy as np  

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

import csv
import os
import numpy as np
from matplotlib import pyplot as plt


Creating and annotating the CSV

In [ ]:
def createCSV(fileName):
    landmarks = ['class']
    for val in range(1,33+1):
        landmarks += ['x{}'.format(val),'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

    with open(fileName, mode='w', newline='')as f:
        csv_writer = csv.writer(f,delimiter = ',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)

# createCSV('deadlift.csv')

def export_landmark(results,action, csvPath):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist() #Converted to a list
        print(keypoints)
        keypoints.insert(0,action)
        
        with open(csvPath, mode='a', newline='') as f:
            csv_writer = csv.writer(f,delimiter = ',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)
        pass

def annotateCSV(videoPath, csvPath):
    cap = cv2.VideoCapture(videoPath) #name of the video
    #Initiate holistic model
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret,frame = cap.read()
            if not ret:
                print("Can't receive frame (stream end?). Exiting ...")
                break
            #Recolor Feed
            image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            #Make detection
            results = pose.process(image)

            #Recolor back to BGR for rendering
            image.flags.writeable = True
            image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

            #Render detections
            mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                    )

            k = cv2.waitKey(1)
            if k== 117:
                export_landmark(results,'up', csvPath)
            elif k== 100:
                export_landmark(results,'down', csvPath)

            cv2.imshow('Mediapipe Feed',image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release() 
    cv2.destroyAllWindows()

def runAnnotation(datasetDir, csvPath):
    # looping through all the training video files
    for file in os.listdir('dataset/deadlift'):
        
        if file.endswith('.mp4'):
            print(file)
            annotateCSV(f"{datasetDir}/{file}", csvPath)

# runAnnotation('dataset/deadlift', 'deadlift.csv')


In [20]:
from moviepy.editor import *

def concatenate(video_clip_paths, output_path, method="compose"):
    """Concatenates several video files into one video file
    and save it to `output_path`. Note that extension (mp4, etc.) must be added to `output_path`
    `method` can be either 'compose' or 'reduce':
        `reduce`: Reduce the quality of the video to the lowest quality on the list of `video_clip_paths`.
        `compose`: type help(concatenate_videoclips) for the info"""
    # create VideoFileClip object for each video file
    clips = [VideoFileClip(c) for c in video_clip_paths]
    if method == "reduce":
        # calculate minimum width & height across all clips
        min_height = min([c.h for c in clips])
        min_width = min([c.w for c in clips])
        # resize the videos to the minimum
        clips = [c.resize(newsize=(min_width, min_height)) for c in clips]
        # concatenate the final video
        final_clip = concatenate_videoclips(clips)
    elif method == "compose":
        # concatenate the final video with the compose method provided by moviepy
        final_clip = concatenate_videoclips(clips, method="compose")
    # write the output video file
    final_clip.write_videofile(output_path)
    final_clip.close()

concatenate("/dataset/deadlift", "singleDeadlift.mp4", method="compose")

In [ ]:
for file in os.listdir('dataset/deadlift'):
    if file.endswith('.mp4'):
        print(file)

Training the model functions

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('annotations/push-up.csv')

# Train Machine Learning Classification Model
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score #Accuracy metrics
import pickle


pipelines = {
    'lr':make_pipeline(StandardScaler(),LogisticRegression()),
    'rc':make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier()),
}

X = df.drop('class', axis=1) #Features
Y = df['class'] #Target value

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.3, random_state = 1234)

fit_models = {}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test.values,yhat),
            precision_score(y_test.values,yhat,average='binary',pos_label='up'),
            recall_score(y_test.values,yhat,average='binary',pos_label='up'))

#dumping the model
with open('models/push-up.pkl','wb') as f:  
    pickle.dump(fit_models['rf'],f)



d:\Shashaank\Clg\MiniProjs\Sem7\AI_Trainer\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr 0.962037962037962 0.9368421052631579 0.9823399558498896
rc 0.9590409590409591 0.9256198347107438 0.9889624724061811
rf 0.9820179820179821 0.9718004338394793 0.9889624724061811
gb 0.968031968031968 0.9469214437367304 0.9845474613686535


In [15]:
# Make Predictions with model
with open('models/push-up.pkl','rb') as f:
    model = pickle.load(f)

cap = cv2.VideoCapture(0)
counter = 0
current_stage = ''
#Initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        #Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detections
        results = pose.process(image)

        #Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Extract landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                )
        landmarks = ['class']
        for val in range(1,33+1):
            landmarks += ['x{}'.format(val),'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row],columns=landmarks[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class,body_language_prob)

            if body_language_class == 'down' and body_language_prob[body_language_prob.argmax()] >= 0.6:
                current_stage = 'down'
            elif current_stage == 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= 0.6:
                current_stage = 'up'
                counter += 1
                print(current_stage,counter)

            #Get status box
            cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1)

            #Display Class
            cv2.putText(image,'CLASS',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,body_language_class.split(' ')[0],(90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Display Probability
            cv2.putText(image,'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(round(body_language_prob[np.argmax(body_language_prob)],2)),(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Display Counter
            cv2.putText(image,'COUNTER',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(counter),(175,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        except Exception as e:
            print(e)
            pass
        
        cv2.imshow('Raw Webcam Feed',image)

        if(cv2.waitKey(10) & 0xFF == ord('q')):
            break

cap.release()
cv2.destroyAllWindows()



'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute

KeyboardInterrupt: 

: 